# Chapter 4 - Exercise 3: Shear Stress and Cell Viability in Extrusion Bioprinting

## Learning Objectives
- Calculate wall shear stress in bioprinting nozzles
- Predict cell viability based on mechanical stress exposure
- Optimize nozzle geometry for cell protection
- Understand trade-offs between resolution and cell survival
- Apply fluid mechanics principles to bioprinting design

## Background

During **extrusion bioprinting**, cells experience mechanical stress as bioink flows through the nozzle. Excessive shear stress causes:
- **Membrane damage** → immediate cell death
- **Intracellular damage** → apoptosis within hours
- **Reduced proliferation** → impaired tissue formation

### Critical Thresholds (from Chapter 4.4.3):
- **τ < 150 Pa**: Inkjet - maintains >85% viability
- **τ < 200 Pa**: Extrusion - maintains >70% viability
- **τ > 500 Pa**: Severe damage for most cell types

### Wall Shear Stress Equation:

For **Newtonian fluids** in cylindrical nozzles:

$$\tau_w = \frac{4\eta Q}{\pi r^3}$$

For **power-law fluids** (shear-thinning bioinks):

$$\tau_w = \left(\frac{3n+1}{4n}\right) \frac{4\eta_a Q}{\pi r^3}$$

where:
- τ_w = wall shear stress (Pa)
- η = dynamic viscosity (Pa·s)
- Q = volumetric flow rate (m³/s)
- r = nozzle radius (m)
- n = flow behavior index

## Setup: Install and Import Libraries

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install numpy matplotlib pandas seaborn scipy plotly -q

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize, fsolve
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

print("✓ All libraries loaded successfully!")
print("Ready to analyze shear stress and cell viability!")

## Part 1: Shear Stress Calculation Functions

In [ ]:
def calculate_wall_shear_stress_newtonian(viscosity, flow_rate, nozzle_diameter):
    """
    Calculate wall shear stress for Newtonian fluids in cylindrical nozzles.
    
    Parameters:
    -----------
    viscosity : float
        Dynamic viscosity η (Pa·s)
    flow_rate : float
        Volumetric flow rate Q (µL/min)
    nozzle_diameter : float
        Nozzle inner diameter d (µm)
    
    Returns:
    --------
    shear_stress : float
        Wall shear stress τ_w (Pa)
    """
    # Convert units
    Q_m3s = flow_rate * 1e-9 / 60  # µL/min → m³/s
    r_m = (nozzle_diameter / 2) * 1e-6  # µm → m
    
    # Calculate wall shear stress
    tau_w = (4 * viscosity * Q_m3s) / (np.pi * r_m**3)
    
    return tau_w

def calculate_wall_shear_stress_powerlaw(K, n, flow_rate, nozzle_diameter):
    """
    Calculate wall shear stress for power-law (shear-thinning) fluids.
    
    Parameters:
    -----------
    K : float
        Consistency index (Pa·s^n)
    n : float
        Flow behavior index (dimensionless)
    flow_rate : float
        Volumetric flow rate Q (µL/min)
    nozzle_diameter : float
        Nozzle inner diameter d (µm)
    
    Returns:
    --------
    shear_stress : float
        Wall shear stress τ_w (Pa)
    """
    # Convert units
    Q_m3s = flow_rate * 1e-9 / 60  # µL/min → m³/s
    r_m = (nozzle_diameter / 2) * 1e-6  # µm → m
    
    # Calculate wall shear rate
    gamma_w = ((3*n + 1) / (4*n)) * (4 * Q_m3s) / (np.pi * r_m**3)
    
    # Calculate wall shear stress
    tau_w = K * gamma_w**n
    
    return tau_w

def predict_cell_viability(shear_stress, cell_type='general'):
    """
    Predict cell viability based on shear stress exposure.
    Based on empirical relationships from literature.
    
    Parameters:
    -----------
    shear_stress : float or array
        Shear stress τ (Pa)
    cell_type : str
        Cell type: 'general', 'fibroblast', 'stem_cell', 'hepatocyte', 'neuron'
    
    Returns:
    --------
    viability : float or array
        Predicted cell viability (0-100%)
    """
    # Define cell-type specific parameters
    cell_params = {
        'general': {'V_max': 95, 'tau_50': 200, 'slope': 0.02},
        'fibroblast': {'V_max': 95, 'tau_50': 250, 'slope': 0.015},  # More robust
        'stem_cell': {'V_max': 90, 'tau_50': 150, 'slope': 0.025},   # Sensitive
        'hepatocyte': {'V_max': 85, 'tau_50': 120, 'slope': 0.03},   # Very sensitive
        'neuron': {'V_max': 80, 'tau_50': 100, 'slope': 0.035}       # Extremely sensitive
    }
    
    params = cell_params.get(cell_type, cell_params['general'])
    V_max = params['V_max']
    tau_50 = params['tau_50']  # Shear stress at 50% viability
    k = params['slope']
    
    # Sigmoidal decay model
    viability = V_max / (1 + np.exp(k * (shear_stress - tau_50)))
    
    # Ensure viability stays in valid range
    viability = np.clip(viability, 0, 100)
    
    return viability

def calculate_print_speed(flow_rate, nozzle_diameter, layer_height):
    """
    Calculate linear print speed based on flow rate and nozzle geometry.
    
    Parameters:
    -----------
    flow_rate : float
        Volumetric flow rate Q (µL/min)
    nozzle_diameter : float
        Nozzle inner diameter d (µm)
    layer_height : float
        Layer height h (µm), typically 70-100% of nozzle diameter
    
    Returns:
    --------
    speed : float
        Linear print speed (mm/s)
    """
    # Convert units
    Q_m3s = flow_rate * 1e-9 / 60  # µL/min → m³/s
    
    # Cross-sectional area of deposited strand (approximate as rectangular)
    width_m = nozzle_diameter * 1e-6  # m
    height_m = layer_height * 1e-6  # m
    area_m2 = width_m * height_m
    
    # Linear speed
    speed_ms = Q_m3s / area_m2
    speed_mms = speed_ms * 1000  # m/s → mm/s
    
    return speed_mms

print("✓ Shear stress calculation functions defined!")
print("\nAvailable functions:")
print("  • calculate_wall_shear_stress_newtonian()")
print("  • calculate_wall_shear_stress_powerlaw()")
print("  • predict_cell_viability()")
print("  • calculate_print_speed()")

## Part 2: Visualize Cell Viability vs Shear Stress

Different cell types have different sensitivities to mechanical stress:

In [ ]:
# Create shear stress range
tau_range = np.linspace(0, 500, 500)

# Calculate viability for different cell types
cell_types = ['fibroblast', 'general', 'stem_cell', 'hepatocyte', 'neuron']
colors_cells = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c', '#9b59b6']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Viability curves
for cell_type, color in zip(cell_types, colors_cells):
    viability = predict_cell_viability(tau_range, cell_type)
    ax1.plot(tau_range, viability, linewidth=2.5, label=cell_type.replace('_', ' ').title(), color=color)

# Add critical thresholds
ax1.axvline(150, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Inkjet limit (150 Pa)')
ax1.axvline(200, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Extrusion limit (200 Pa)')
ax1.axhline(85, color='green', linestyle=':', linewidth=2, alpha=0.5, label='Target viability (85%)')

# Add safe zone
ax1.axvspan(0, 150, alpha=0.1, color='green', label='Safe zone')
ax1.axvspan(200, 500, alpha=0.1, color='red', label='Danger zone')

ax1.set_xlabel('Wall Shear Stress, τ (Pa)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Cell Viability (%)', fontsize=12, fontweight='bold')
ax1.set_title('Cell Viability vs Shear Stress\n(Cell-Type Specific)', fontsize=13, fontweight='bold')
ax1.legend(loc='upper right', frameon=True, fancybox=True, fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 500)
ax1.set_ylim(0, 100)

# Plot 2: Critical stress values
cell_labels = [ct.replace('_', '\n').title() for ct in cell_types]
tau_50_values = [250, 200, 150, 120, 100]  # From predict_cell_viability function

bars = ax2.barh(cell_labels, tau_50_values, color=colors_cells, alpha=0.7, edgecolor='black', linewidth=1.5)
ax2.axvline(200, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Extrusion limit')
ax2.axvline(150, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Inkjet limit')

# Add value labels
for bar, value in zip(bars, tau_50_values):
    ax2.text(value + 10, bar.get_y() + bar.get_height()/2, f'{value} Pa',
            va='center', fontweight='bold', fontsize=10)

ax2.set_xlabel('Shear Stress at 50% Viability (Pa)', fontsize=12, fontweight='bold')
ax2.set_title('Cell Type Sensitivity Ranking\n(Higher = More Robust)', fontsize=13, fontweight='bold')
ax2.legend(loc='lower right', frameon=True, fancybox=True)
ax2.grid(True, alpha=0.3, axis='x')
ax2.set_xlim(0, 300)

plt.tight_layout()
plt.show()

print("\n💡 Key Observations:")
print("   • Fibroblasts: Most robust (τ_50 = 250 Pa)")
print("   • Neurons: Most sensitive (τ_50 = 100 Pa)")
print("   • Hepatocytes: Very sensitive (τ_50 = 120 Pa)")
print("   • All cells: Viability drops sharply above 200 Pa")
print("   • Target: Keep τ < 150 Pa for >85% viability across all cell types")

## Part 3: Interactive Nozzle Design Calculator

### 🎯 STUDENT TASK 1: Design a Nozzle for Optimal Cell Viability

**Modify the parameters below to explore the design space:**

In [ ]:
# ==========================================
# STUDENT PARAMETERS - MODIFY THESE VALUES
# ==========================================

# Bioink properties
bioink_viscosity = 0.1      # Pa·s (try: 0.05, 0.1, 0.5, 1.0)
                            # Note: 0.1 Pa·s = 100 mPa·s

# Nozzle geometry
nozzle_diameter = 250       # µm (try: 100, 200, 250, 400)

# Printing parameters
flow_rate = 50              # µL/min (try: 10, 30, 50, 100)
layer_height = 200          # µm (typically 80% of nozzle diameter)

# Cell type
selected_cell_type = 'stem_cell'  # Options: 'fibroblast', 'general', 'stem_cell', 'hepatocyte', 'neuron'

# ==========================================

# Calculate shear stress
tau_wall = calculate_wall_shear_stress_newtonian(bioink_viscosity, flow_rate, nozzle_diameter)

# Predict cell viability
viability = predict_cell_viability(tau_wall, selected_cell_type)

# Calculate print speed
print_speed = calculate_print_speed(flow_rate, nozzle_diameter, layer_height)

# Calculate average shear rate
Q_m3s = flow_rate * 1e-9 / 60
r_m = (nozzle_diameter / 2) * 1e-6
shear_rate_avg = (4 * Q_m3s) / (np.pi * r_m**3)

# Create comprehensive visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Nozzle Design', 'Viability Assessment', 
                   'Stress Distribution', 'Performance Summary'),
    specs=[[{'type': 'scatter'}, {'type': 'indicator'}],
           [{'type': 'scatter'}, {'type': 'table'}]]
)

# Plot 1: Nozzle schematic
nozzle_x = [0, 0, nozzle_diameter/2, nozzle_diameter/2, 0, 0, -nozzle_diameter/2, -nozzle_diameter/2, 0]
nozzle_y = [300, 200, 200, 0, 0, 200, 200, 300, 300]

fig.add_trace(
    go.Scatter(x=nozzle_x, y=nozzle_y, fill='toself',
               fillcolor='lightblue', line=dict(color='black', width=2),
               name='Nozzle', showlegend=False),
    row=1, col=1
)

# Add dimensions
fig.add_annotation(x=0, y=100, text=f"d = {nozzle_diameter} µm",
                  showarrow=False, font=dict(size=12, color='red'),
                  row=1, col=1)

fig.update_xaxes(title_text="Position (µm)", row=1, col=1)
fig.update_yaxes(title_text="Height (µm)", row=1, col=1)

# Plot 2: Viability gauge
if viability >= 85:
    gauge_color = "green"
    assessment = "Excellent"
elif viability >= 75:
    gauge_color = "yellow"
    assessment = "Acceptable"
elif viability >= 60:
    gauge_color = "orange"
    assessment = "Marginal"
else:
    gauge_color = "red"
    assessment = "Poor"

fig.add_trace(
    go.Indicator(
        mode="gauge+number+delta",
        value=viability,
        title={'text': f"Cell Viability<br>{assessment}"},
        delta={'reference': 85, 'valueformat': '.1f'},
        gauge={
            'axis': {'range': [None, 100]},
            'bar': {'color': gauge_color},
            'steps': [
                {'range': [0, 60], 'color': "lightgray"},
                {'range': [60, 75], 'color': "lightyellow"},
                {'range': [75, 85], 'color': "lightgreen"},
                {'range': [85, 100], 'color': "green"}],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 85}
        }),
    row=1, col=2
)

# Plot 3: Stress profile across radius
r_profile = np.linspace(0, nozzle_diameter/2, 100)
tau_profile = tau_wall * (r_profile / (nozzle_diameter/2))  # Linear profile for Newtonian

fig.add_trace(
    go.Scatter(x=r_profile, y=tau_profile,
               mode='lines', name='Shear stress',
               line=dict(color='red', width=3)),
    row=2, col=1
)

# Add critical threshold
fig.add_hline(y=200, line_dash="dash", line_color="orange",
             annotation_text="Critical (200 Pa)", row=2, col=1)

fig.update_xaxes(title_text="Radial Position (µm)", row=2, col=1)
fig.update_yaxes(title_text="Shear Stress (Pa)", row=2, col=1)

# Plot 4: Performance table
fig.add_trace(
    go.Table(
        header=dict(values=['Parameter', 'Value'],
                   fill_color='paleturquoise',
                   align='left',
                   font=dict(size=12, color='black')),
        cells=dict(values=[
            ['Wall Shear Stress', 'Cell Viability', 'Print Speed', 'Shear Rate', 'Resolution'],
            [f'{tau_wall:.1f} Pa', f'{viability:.1f}%', f'{print_speed:.2f} mm/s', 
             f'{shear_rate_avg:.0f} 1/s', f'{nozzle_diameter} µm']
        ],
        fill_color='lavender',
        align='left',
        font=dict(size=11))),
    row=2, col=2
)

fig.update_layout(height=800, showlegend=False,
                 title_text=f"Nozzle Performance Analysis - {selected_cell_type.replace('_', ' ').title()}")
fig.show()

# Detailed text output
print("="*70)
print("NOZZLE DESIGN ANALYSIS")
print("="*70)
print(f"\nInput Parameters:")
print(f"  • Bioink viscosity: {bioink_viscosity*1000:.0f} mPa·s")
print(f"  • Nozzle diameter: {nozzle_diameter} µm")
print(f"  • Flow rate: {flow_rate} µL/min")
print(f"  • Cell type: {selected_cell_type.replace('_', ' ').title()}")

print(f"\nCalculated Results:")
print(f"  • Wall shear stress: {tau_wall:.1f} Pa")
print(f"  • Average shear rate: {shear_rate_avg:.0f} 1/s")
print(f"  • Predicted viability: {viability:.1f}%")
print(f"  • Print speed: {print_speed:.2f} mm/s")

print(f"\nAssessment:")
if tau_wall < 150:
    print(f"  ✅ Excellent: τ = {tau_wall:.1f} Pa < 150 Pa")
    print("  • Suitable for all cell types including sensitive neurons")
    print("  • Expected viability >85%")
elif tau_wall < 200:
    print(f"  ✓ Good: 150 Pa < τ = {tau_wall:.1f} Pa < 200 Pa")
    print("  • Acceptable for robust cells (fibroblasts, MSCs)")
    print("  • May damage sensitive cells (neurons, hepatocytes)")
elif tau_wall < 300:
    print(f"  ⚠️  Marginal: 200 Pa < τ = {tau_wall:.1f} Pa < 300 Pa")
    print("  • Significant cell damage expected")
    print("  • Viability likely <70%")
    print("  • Consider: larger nozzle, lower flow rate, or lower viscosity")
else:
    print(f"  ❌ Poor: τ = {tau_wall:.1f} Pa > 300 Pa")
    print("  • Severe cell damage")
    print("  • Viability likely <50%")
    print("  • Redesign required!")

if viability >= 85:
    print(f"\n✅ Cell viability: {viability:.1f}% - Excellent!")
elif viability >= 70:
    print(f"\n✓ Cell viability: {viability:.1f}% - Acceptable")
else:
    print(f"\n❌ Cell viability: {viability:.1f}% - Unacceptable")

print(f"\n💡 Optimization Suggestions:")
if tau_wall > 200:
    # Calculate required changes
    target_tau = 180  # Target below 200 Pa
    reduction_factor = target_tau / tau_wall
    
    new_diameter = nozzle_diameter / (reduction_factor**(1/3))
    new_flow_rate = flow_rate * reduction_factor
    new_viscosity = bioink_viscosity * reduction_factor
    
    print(f"  To reduce stress to ~{target_tau} Pa:")
    print(f"    • Increase nozzle diameter to {new_diameter:.0f} µm (current: {nozzle_diameter} µm)")
    print(f"    OR reduce flow rate to {new_flow_rate:.1f} µL/min (current: {flow_rate} µL/min)")
    print(f"    OR reduce viscosity to {new_viscosity*1000:.0f} mPa·s (current: {bioink_viscosity*1000:.0f} mPa·s)")
else:
    print(f"  ✓ Current design is suitable!")
    if nozzle_diameter > 200:
        print(f"  • Could use smaller nozzle for better resolution if needed")
    if print_speed < 5:
        print(f"  • Could increase flow rate for faster printing")

print("\n" + "="*70)

## Part 4: Parameter Sweep - Design Space Exploration

Explore how nozzle diameter and flow rate affect both cell viability and print speed:

In [ ]:
# Create parameter ranges
nozzle_diameters = np.linspace(100, 400, 50)  # µm
flow_rates = np.linspace(10, 100, 50)  # µL/min

# Create meshgrid
D, Q = np.meshgrid(nozzle_diameters, flow_rates)

# Calculate shear stress for each combination
viscosity_fixed = 0.1  # Pa·s (100 mPa·s - typical GelMA)
tau_grid = np.zeros_like(D)

for i in range(D.shape[0]):
    for j in range(D.shape[1]):
        tau_grid[i, j] = calculate_wall_shear_stress_newtonian(
            viscosity_fixed, Q[i, j], D[i, j]
        )

# Calculate viability
viability_grid = predict_cell_viability(tau_grid, 'general')

# Calculate print speed
speed_grid = np.zeros_like(D)
layer_height_factor = 0.8  # 80% of nozzle diameter

for i in range(D.shape[0]):
    for j in range(D.shape[1]):
        speed_grid[i, j] = calculate_print_speed(
            Q[i, j], D[i, j], D[i, j] * layer_height_factor
        )

# Create figure with subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 14))

# Plot 1: Shear stress contour
contour1 = ax1.contourf(D, Q, tau_grid, levels=20, cmap='YlOrRd')
contour1_lines = ax1.contour(D, Q, tau_grid, levels=[150, 200, 300], 
                             colors=['yellow', 'orange', 'red'], linewidths=2)
ax1.clabel(contour1_lines, inline=True, fontsize=10, fmt='%d Pa')
cbar1 = plt.colorbar(contour1, ax=ax1)
cbar1.set_label('Wall Shear Stress (Pa)', fontsize=11, fontweight='bold')

ax1.set_xlabel('Nozzle Diameter (µm)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Flow Rate (µL/min)', fontsize=12, fontweight='bold')
ax1.set_title('Wall Shear Stress Map\n(Viscosity = 100 mPa·s)', fontsize=13, fontweight='bold')

# Plot 2: Viability contour
contour2 = ax2.contourf(D, Q, viability_grid, levels=20, cmap='RdYlGn')
contour2_lines = ax2.contour(D, Q, viability_grid, levels=[70, 80, 85, 90], 
                             colors=['red', 'orange', 'yellow', 'green'], linewidths=2)
ax2.clabel(contour2_lines, inline=True, fontsize=10, fmt='%d%%')
cbar2 = plt.colorbar(contour2, ax=ax2)
cbar2.set_label('Cell Viability (%)', fontsize=11, fontweight='bold')

ax2.set_xlabel('Nozzle Diameter (µm)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Flow Rate (µL/min)', fontsize=12, fontweight='bold')
ax2.set_title('Cell Viability Map\n(General Cell Type)', fontsize=13, fontweight='bold')

# Plot 3: Print speed contour
contour3 = ax3.contourf(D, Q, speed_grid, levels=20, cmap='viridis')
contour3_lines = ax3.contour(D, Q, speed_grid, levels=[5, 10, 20, 30], 
                             colors='white', linewidths=1.5, linestyles='dashed')
ax3.clabel(contour3_lines, inline=True, fontsize=10, fmt='%d mm/s')
cbar3 = plt.colorbar(contour3, ax=ax3)
cbar3.set_label('Print Speed (mm/s)', fontsize=11, fontweight='bold')

ax3.set_xlabel('Nozzle Diameter (µm)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Flow Rate (µL/min)', fontsize=12, fontweight='bold')
ax3.set_title('Print Speed Map\n(Layer Height = 80% of Diameter)', fontsize=13, fontweight='bold')

# Plot 4: Safe operating zone
# Create mask for acceptable conditions (viability > 85% AND speed > 5 mm/s)
safe_zone = (viability_grid >= 85) & (speed_grid >= 5)
acceptable_zone = (viability_grid >= 75) & (viability_grid < 85) & (speed_grid >= 5)

ax4.contourf(D, Q, safe_zone.astype(float), levels=[0.5, 1.5], 
            colors=['white', 'lightgreen'], alpha=0.6)
ax4.contourf(D, Q, acceptable_zone.astype(float), levels=[0.5, 1.5],
            colors=['white', 'lightyellow'], alpha=0.6)

# Add boundary lines
ax4.contour(D, Q, viability_grid, levels=[85], colors=['green'], linewidths=3, linestyles='solid')
ax4.contour(D, Q, viability_grid, levels=[75], colors=['orange'], linewidths=2, linestyles='dashed')

ax4.set_xlabel('Nozzle Diameter (µm)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Flow Rate (µL/min)', fontsize=12, fontweight='bold')
ax4.set_title('Safe Operating Zone\n(Green: Viability ≥85%, Yellow: 75-85%)', 
             fontsize=13, fontweight='bold')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='lightgreen', edgecolor='green', linewidth=2, label='Excellent (≥85%)'),
    Patch(facecolor='lightyellow', edgecolor='orange', linewidth=2, label='Acceptable (75-85%)'),
    Patch(facecolor='white', edgecolor='red', linewidth=2, label='Poor (<75%)')
]
ax4.legend(handles=legend_elements, loc='upper right', frameon=True, fancybox=True)

plt.tight_layout()
plt.show()

print("\n💡 Design Space Analysis:")
print("   • Larger nozzles → Lower shear stress → Higher viability")
print("   • Higher flow rates → Higher shear stress → Lower viability")
print("   • Trade-off: Resolution vs Cell Protection")
print("   • Optimal zone: 250-350 µm diameter, 20-60 µL/min flow")
print("   • This achieves >85% viability with 5-15 mm/s print speed")

## Part 5: Power-Law Bioink Comparison

### 🎯 STUDENT TASK 2: Compare Newtonian vs Shear-Thinning Bioinks

**See how shear-thinning reduces cell damage:**

In [ ]:
# ==========================================
# STUDENT PARAMETERS - MODIFY THESE VALUES
# ==========================================

# Bioink A: Newtonian (e.g., dilute solution)
viscosity_newtonian = 0.1  # Pa·s

# Bioink B: Power-law (e.g., GelMA, alginate-gelatin)
K_powerlaw = 0.5           # Pa·s^n (try: 0.3, 0.5, 1.0, 2.0)
n_powerlaw = 0.5           # dimensionless (try: 0.4, 0.5, 0.6, 0.7)

# Printing conditions
nozzle_d_compare = 250     # µm
flow_rate_compare = 50     # µL/min

# ==========================================

# Calculate for Newtonian bioink
tau_newtonian = calculate_wall_shear_stress_newtonian(
    viscosity_newtonian, flow_rate_compare, nozzle_d_compare
)
viability_newtonian = predict_cell_viability(tau_newtonian, 'general')

# Calculate for power-law bioink
tau_powerlaw = calculate_wall_shear_stress_powerlaw(
    K_powerlaw, n_powerlaw, flow_rate_compare, nozzle_d_compare
)
viability_powerlaw = predict_cell_viability(tau_powerlaw, 'general')

# Create comparison visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Viscosity profiles
shear_rates_profile = np.logspace(0, 3, 200)
visc_newtonian_profile = np.ones_like(shear_rates_profile) * viscosity_newtonian * 1000
visc_powerlaw_profile = K_powerlaw * shear_rates_profile**(n_powerlaw - 1) * 1000

ax1.loglog(shear_rates_profile, visc_newtonian_profile, linewidth=3, 
          label='Newtonian', color='#3498db')
ax1.loglog(shear_rates_profile, visc_powerlaw_profile, linewidth=3,
          label=f'Power-law (n={n_powerlaw})', color='#e74c3c')

# Mark typical extrusion shear rate
Q_m3s = flow_rate_compare * 1e-9 / 60
r_m = (nozzle_d_compare / 2) * 1e-6
gamma_typical = (4 * Q_m3s) / (np.pi * r_m**3)
ax1.axvline(gamma_typical, color='green', linestyle='--', linewidth=2, 
           label=f'Extrusion shear rate ({gamma_typical:.0f} 1/s)')

ax1.set_xlabel('Shear Rate (1/s)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Viscosity (mPa·s)', fontsize=12, fontweight='bold')
ax1.set_title('Viscosity Comparison', fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True)
ax1.grid(True, alpha=0.3, which='both')

# Plot 2: Stress comparison
categories = ['Newtonian', 'Power-law']
stress_values = [tau_newtonian, tau_powerlaw]
colors_bar = ['#3498db', '#e74c3c']

bars = ax2.bar(categories, stress_values, color=colors_bar, alpha=0.7, 
              edgecolor='black', linewidth=2)
ax2.axhline(200, color='red', linestyle='--', linewidth=2, label='Safety limit (200 Pa)')
ax2.axhline(150, color='orange', linestyle='--', linewidth=2, label='Optimal (<150 Pa)')

# Add value labels
for bar, value in zip(bars, stress_values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.1f} Pa',
            ha='center', va='bottom', fontweight='bold', fontsize=12)

ax2.set_ylabel('Wall Shear Stress (Pa)', fontsize=12, fontweight='bold')
ax2.set_title('Shear Stress Comparison', fontsize=13, fontweight='bold')
ax2.legend(loc='upper right', frameon=True, fancybox=True)
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Viability comparison
viability_values = [viability_newtonian, viability_powerlaw]

bars2 = ax3.bar(categories, viability_values, color=colors_bar, alpha=0.7,
               edgecolor='black', linewidth=2)
ax3.axhline(85, color='green', linestyle='--', linewidth=2, label='Target (85%)')

# Add value labels
for bar, value in zip(bars2, viability_values):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.1f}%',
            ha='center', va='bottom', fontweight='bold', fontsize=12)

ax3.set_ylabel('Cell Viability (%)', fontsize=12, fontweight='bold')
ax3.set_ylim(0, 100)
ax3.set_title('Predicted Cell Viability', fontsize=13, fontweight='bold')
ax3.legend(loc='lower right', frameon=True, fancybox=True)
ax3.grid(True, alpha=0.3, axis='y')

# Plot 4: Stress reduction benefit
stress_reduction = ((tau_newtonian - tau_powerlaw) / tau_newtonian) * 100
viability_improvement = viability_powerlaw - viability_newtonian

metrics = ['Stress Reduction\n(%)', 'Viability Gain\n(percentage points)']
values_improvement = [stress_reduction, viability_improvement]
colors_improvement = ['#f39c12', '#2ecc71']

bars3 = ax4.bar(metrics, values_improvement, color=colors_improvement, alpha=0.7,
               edgecolor='black', linewidth=2)

# Add value labels
for bar, value in zip(bars3, values_improvement):
    height = bar.get_height()
    if height >= 0:
        va_pos = 'bottom'
        y_pos = height
    else:
        va_pos = 'top'
        y_pos = height
    ax4.text(bar.get_x() + bar.get_width()/2., y_pos,
            f'{value:+.1f}',
            ha='center', va=va_pos, fontweight='bold', fontsize=12)

ax4.axhline(0, color='black', linewidth=1)
ax4.set_ylabel('Improvement', fontsize=12, fontweight='bold')
ax4.set_title('Shear-Thinning Benefit', fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Detailed comparison
print("="*70)
print("NEWTONIAN vs POWER-LAW BIOINK COMPARISON")
print("="*70)
print(f"\nPrinting Conditions:")
print(f"  • Nozzle diameter: {nozzle_d_compare} µm")
print(f"  • Flow rate: {flow_rate_compare} µL/min")

print(f"\nBioink A - Newtonian:")
print(f"  • Viscosity: {viscosity_newtonian*1000:.0f} mPa·s (constant)")
print(f"  • Wall shear stress: {tau_newtonian:.1f} Pa")
print(f"  • Predicted viability: {viability_newtonian:.1f}%")

print(f"\nBioink B - Power-law:")
print(f"  • K = {K_powerlaw} Pa·s^n, n = {n_powerlaw}")
print(f"  • Viscosity at {gamma_typical:.0f} 1/s: {K_powerlaw * gamma_typical**(n_powerlaw-1)*1000:.0f} mPa·s")
print(f"  • Wall shear stress: {tau_powerlaw:.1f} Pa")
print(f"  • Predicted viability: {viability_powerlaw:.1f}%")

print(f"\nBenefit of Shear-Thinning:")
if stress_reduction > 0:
    print(f"  ✅ Stress reduced by {stress_reduction:.1f}%")
    print(f"  ✅ Viability improved by {viability_improvement:.1f} percentage points")
    print(f"\n💡 Shear-thinning is beneficial!")
    print(f"     Lower n → stronger shear-thinning → better cell protection")
else:
    print(f"  ⚠️  Power-law bioink has HIGHER stress (unusual)")
    print(f"     This suggests K is too high or n is too close to 1")

print("\n" + "="*70)

## Part 6: Multi-Objective Optimization

Find the optimal balance between resolution, cell viability, and print speed:

In [ ]:
from scipy.optimize import differential_evolution

def objective_function(params, weights):
    """
    Multi-objective optimization function.
    Minimizes negative weighted sum of normalized objectives.
    
    Parameters:
    -----------
    params : array
        [nozzle_diameter (µm), flow_rate (µL/min)]
    weights : dict
        Weights for each objective: 'viability', 'resolution', 'speed'
    
    Returns:
    --------
    score : float
        Negative weighted objective (for minimization)
    """
    nozzle_d, flow_rate = params
    
    # Fixed parameters
    viscosity = 0.1  # Pa·s
    layer_height = nozzle_d * 0.8
    
    # Calculate metrics
    tau = calculate_wall_shear_stress_newtonian(viscosity, flow_rate, nozzle_d)
    viability = predict_cell_viability(tau, 'general')
    speed = calculate_print_speed(flow_rate, nozzle_d, layer_height)
    resolution_score = 400 - nozzle_d  # Smaller = better (max 400)
    
    # Normalize to 0-1 range
    viability_norm = viability / 100
    speed_norm = min(speed / 30, 1)  # Cap at 30 mm/s
    resolution_norm = resolution_score / 300  # 100-400 µm range
    
    # Weighted sum
    score = (
        weights['viability'] * viability_norm +
        weights['resolution'] * resolution_norm +
        weights['speed'] * speed_norm
    )
    
    return -score  # Negative for minimization

# Define optimization scenarios
scenarios = {
    'Cell Protection Priority': {'viability': 0.7, 'resolution': 0.2, 'speed': 0.1},
    'Balanced': {'viability': 0.4, 'resolution': 0.3, 'speed': 0.3},
    'Resolution Priority': {'viability': 0.2, 'resolution': 0.6, 'speed': 0.2},
    'Speed Priority': {'viability': 0.3, 'resolution': 0.2, 'speed': 0.5}
}

# Optimize for each scenario
bounds = [(100, 400), (10, 100)]  # [nozzle_diameter, flow_rate]
results = {}

print("Optimizing nozzle designs...\n")

for scenario_name, weights in scenarios.items():
    result = differential_evolution(
        objective_function,
        bounds,
        args=(weights,),
        seed=42,
        maxiter=100,
        atol=0.01,
        tol=0.01
    )
    
    optimal_d, optimal_q = result.x
    
    # Calculate final metrics
    tau_opt = calculate_wall_shear_stress_newtonian(0.1, optimal_q, optimal_d)
    viab_opt = predict_cell_viability(tau_opt, 'general')
    speed_opt = calculate_print_speed(optimal_q, optimal_d, optimal_d * 0.8)
    
    results[scenario_name] = {
        'diameter': optimal_d,
        'flow_rate': optimal_q,
        'shear_stress': tau_opt,
        'viability': viab_opt,
        'speed': speed_opt
    }

# Create visualization of results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

scenario_names = list(results.keys())
colors_scenarios = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']

# Plot 1: Nozzle diameter
diameters = [results[s]['diameter'] for s in scenario_names]
bars1 = ax1.barh(scenario_names, diameters, color=colors_scenarios, alpha=0.7, 
                edgecolor='black', linewidth=1.5)
for bar, value in zip(bars1, diameters):
    ax1.text(value + 10, bar.get_y() + bar.get_height()/2, f'{value:.0f} µm',
            va='center', fontweight='bold')
ax1.set_xlabel('Nozzle Diameter (µm)', fontsize=12, fontweight='bold')
ax1.set_title('Optimal Nozzle Diameter', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

# Plot 2: Cell viability
viabilities = [results[s]['viability'] for s in scenario_names]
bars2 = ax2.barh(scenario_names, viabilities, color=colors_scenarios, alpha=0.7,
                edgecolor='black', linewidth=1.5)
ax2.axvline(85, color='green', linestyle='--', linewidth=2, label='Target')
for bar, value in zip(bars2, viabilities):
    ax2.text(value + 2, bar.get_y() + bar.get_height()/2, f'{value:.1f}%',
            va='center', fontweight='bold')
ax2.set_xlabel('Cell Viability (%)', fontsize=12, fontweight='bold')
ax2.set_xlim(70, 100)
ax2.set_title('Predicted Cell Viability', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='x')

# Plot 3: Print speed
speeds = [results[s]['speed'] for s in scenario_names]
bars3 = ax3.barh(scenario_names, speeds, color=colors_scenarios, alpha=0.7,
                edgecolor='black', linewidth=1.5)
for bar, value in zip(bars3, speeds):
    ax3.text(value + 0.5, bar.get_y() + bar.get_height()/2, f'{value:.1f} mm/s',
            va='center', fontweight='bold')
ax3.set_xlabel('Print Speed (mm/s)', fontsize=12, fontweight='bold')
ax3.set_title('Linear Print Speed', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='x')

# Plot 4: Radar chart comparison
from math import pi

categories = ['Viability', 'Resolution\n(inverted)', 'Speed']
N = len(categories)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

ax4 = plt.subplot(224, projection='polar')

for scenario_name, color in zip(scenario_names, colors_scenarios):
    r = results[scenario_name]
    values = [
        r['viability'] / 100,  # Normalize to 0-1
        (400 - r['diameter']) / 300,  # Smaller diameter = better resolution
        min(r['speed'] / 30, 1)  # Normalize speed
    ]
    values += values[:1]
    
    ax4.plot(angles, values, 'o-', linewidth=2, label=scenario_name, color=color)
    ax4.fill(angles, values, alpha=0.15, color=color)

ax4.set_xticks(angles[:-1])
ax4.set_xticklabels(categories, fontsize=10)
ax4.set_ylim(0, 1)
ax4.set_title('Multi-Objective Comparison\n(Normalized 0-1)', 
             fontsize=13, fontweight='bold', pad=20)
ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), frameon=True, fancybox=True)
ax4.grid(True)

plt.tight_layout()
plt.show()

# Print detailed results
print("\n" + "="*80)
print("MULTI-OBJECTIVE OPTIMIZATION RESULTS")
print("="*80)

for scenario_name, r in results.items():
    print(f"\n{scenario_name}:")
    print(f"  Optimal Design:")
    print(f"    • Nozzle diameter: {r['diameter']:.0f} µm")
    print(f"    • Flow rate: {r['flow_rate']:.1f} µL/min")
    print(f"  Performance:")
    print(f"    • Shear stress: {r['shear_stress']:.1f} Pa")
    print(f"    • Cell viability: {r['viability']:.1f}%")
    print(f"    • Print speed: {r['speed']:.2f} mm/s")

print("\n" + "="*80)
print("\n💡 Recommendations:")
print("   • Cell-sensitive applications (neurons, hepatocytes):")
print("     → Use 'Cell Protection Priority' design")
print("   • High-resolution vascular networks:")
print("     → Use 'Resolution Priority' design")
print("   • Large-scale tissue constructs:")
print("     → Use 'Speed Priority' design")
print("   • General bioprinting:")
print("     → Use 'Balanced' design")

## Part 7: Summary and Key Takeaways

In [ ]:
print("="*80)
print("CHAPTER 4 EXERCISE 3: KEY LEARNING POINTS")
print("="*80)
print("""
1. SHEAR STRESS IS THE PRIMARY CAUSE OF CELL DAMAGE
   → Wall shear stress τ_w scales as: τ_w ∝ η·Q/r³
   → Cubic dependence on radius means small diameter changes have huge impact
   → Doubling nozzle diameter reduces stress by 8×!

2. CRITICAL STRESS THRESHOLDS (FROM CHAPTER 4)
   → τ < 150 Pa: Safe for all cell types (>85% viability)
   → τ = 150-200 Pa: Acceptable for robust cells (fibroblasts, MSCs)
   → τ = 200-300 Pa: Marginal - significant damage
   → τ > 300 Pa: Severe damage (<50% viability)

3. CELL-TYPE SENSITIVITY RANKING
   → Fibroblasts: Most robust (τ_50 = 250 Pa)
   → Mesenchymal stem cells: Moderate (τ_50 = 200 Pa)
   → Stem cells: Sensitive (τ_50 = 150 Pa)
   → Hepatocytes: Very sensitive (τ_50 = 120 Pa)
   → Neurons: Extremely sensitive (τ_50 = 100 Pa)

4. NOZZLE DESIGN TRADE-OFFS
   → Resolution vs Cell Protection:
     • Small nozzle (100 µm) = high resolution BUT high stress
     • Large nozzle (400 µm) = low stress BUT low resolution
   → Optimal range for most applications: 200-300 µm

5. SHEAR-THINNING REDUCES CELL DAMAGE
   → Power-law bioinks (n < 1) have lower viscosity at high shear rates
   → This reduces stress during extrusion while maintaining shape retention
   → Strong shear-thinning (n = 0.4-0.5) can reduce stress by 40-60%
   → Examples: GelMA, alginate-gelatin, nanocellulose composites

6. OPTIMIZATION STRATEGIES
   → Increase nozzle diameter (most effective)
   → Reduce flow rate (but decreases print speed)
   → Use shear-thinning bioinks (n < 0.6)
   → Lower bioink viscosity (but may reduce shape fidelity)
   → Consider multi-pass printing with lower flow per pass

7. MULTI-OBJECTIVE DESIGN REQUIRES TRADE-OFFS
   → Cannot maximize viability, resolution, AND speed simultaneously
   → Must prioritize based on application:
     • Vascular networks → prioritize resolution
     • Sensitive cells → prioritize viability
     • Large constructs → prioritize speed

8. PRACTICAL DESIGN GUIDELINES
   → For fibroblasts/MSCs: 200-250 µm nozzle OK
   → For hepatocytes: 250-350 µm nozzle recommended
   → For neurons: 300-400 µm nozzle + shear-thinning bioink essential
   → Target flow rates: 20-60 µL/min for most applications
   → Always verify: Calculate τ_w before printing!
""")
print("="*80)

## 🎓 REFLECTION QUESTIONS

### Question 1
**Explain why wall shear stress has a cubic dependence on nozzle radius (τ ∝ 1/r³). What does this mean for practical nozzle design?**

### Question 2  
**You need to print hepatocytes (very sensitive, τ_50 = 120 Pa) with your current setup: 200 µm nozzle, 60 µL/min flow, η = 0.15 Pa·s. Calculate the shear stress and viability. What changes would you make?**

### Question 3
**Compare two bioinks at 50 µL/min through a 250 µm nozzle:**
**- Bioink A: η = 0.1 Pa·s (Newtonian)**
**- Bioink B: K = 0.5 Pa·s^n, n = 0.5 (power-law)**
**Which causes less cell damage and why?**

### Question 4
**Your lab needs to print a vascular network (requires 100 µm resolution) using neural stem cells (very shear-sensitive). Describe the challenges and propose a multi-step solution.**

### Question 5
**Why do larger nozzles not only reduce shear stress but also enable higher print speeds? Explain the relationship between nozzle diameter, flow rate, and print speed.**

## 📚 Additional Challenges (Optional)

### Challenge 1: Non-Circular Nozzles
Research rectangular nozzles. How does aspect ratio affect shear stress distribution? When might rectangular nozzles be advantageous?

### Challenge 2: Pulsatile Flow
Model time-varying flow rate (pulsatile extrusion). How does this affect peak shear stress vs continuous flow?

### Challenge 3: Coaxial Nozzles
For coaxial bioprinting (core-shell), calculate shear stress in both the inner and outer channels. How does this change the design constraints?

### Challenge 4: Experimental Validation
Design an experiment to measure actual cell viability vs predicted values. What controls would you include? What are potential sources of error?

## 🎯 Congratulations!

You've completed Exercise 3: Shear Stress and Cell Viability!

**You now understand:**
- ✓ How to calculate wall shear stress in bioprinting nozzles
- ✓ Cell-type specific sensitivity to mechanical stress
- ✓ Trade-offs between resolution, viability, and print speed
- ✓ Benefits of shear-thinning bioinks for cell protection
- ✓ Multi-objective optimization for nozzle design
- ✓ Practical design guidelines for different cell types

**Next Steps:**
- Continue to Exercise 4: Inkjet Droplet Formation
- Review Chapter 4.4.3 for deeper understanding of extrusion bioprinting
- Explore research papers on cell mechanobiology and bioprinting

---

*Exercise created for Biofabrication Chapter 4 - Master's Level Bioengineering*